In [1]:
from eventvision import read_dataset, read_annotation

import os
import cv2
import glob
import time
import numpy as np
import contextlib
import torch.cuda.amp
from sklearn.model_selection import train_test_split

import spconv.pytorch as spconv
import torch
import torch.optim as optim
from torch import nn
import torch.nn.functional as F

from BoundingBoxRegressionUtils import parse_partial_dataset, parse_random_dataset, torch_iou


POINTS = 500

Event-based vision module imported


In [2]:
def getData(filename):
    td = read_dataset(filename)
    
    width = td.width
    height = td.height
    
    points = np.column_stack((td.data.x[:POINTS], td.data.y[:POINTS], td.data.ts[:POINTS] / 1000))
    
    return width, height, np.array(points).reshape(POINTS, 3, 1)

def getAnnotation(filename):
    data = read_annotation(filename)
    minX = min(data[0])
    maxX = max(data[0])
    minY = min(data[1])
    maxY = max(data[1])
    
    return minX, minY, maxX, maxY

In [3]:
def train(args, model, device, train_loader, optimizer, epoch):
    model.train()
    correct = 0
    amp_ctx = contextlib.nullcontext()
    if args.fp16:
        amp_ctx = torch.cuda.amp.autocast()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        with amp_ctx:
            output = model(data)
            loss = F.mse_loss(output, target)
            loss.backward()
            optimizer.step()
            
            correct += torch.sum(torch_iou(output[:, -4:], target[:, -4:]))
            
    return correct / len(train_loader.dataset)

def test(args, model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    amp_ctx = contextlib.nullcontext()
    if args.fp16:
        amp_ctx = torch.cuda.amp.autocast()

    with torch.no_grad():
        for data, target in test_loader:

            data, target = data.to(device), target.to(device)
            with amp_ctx:

                output = model(data)
            test_loss += F.mse_loss(output, target, reduction='sum').item() # sum up batch loss
            
            correct += torch.sum(torch_iou(output[:, -4:], target[:, -4:]))
            

    test_loss /= len(test_loader.dataset)

    
    return correct / len(test_loader.dataset)

In [4]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.net = spconv.SparseSequential(
            spconv.SparseConv2d(1, 16, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2),
            
            spconv.SparseConv2d(16, 32, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2, 1),
            
            spconv.SparseConv2d(32, 64, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2, 1),
            
            spconv.SparseConv2d(64, 128, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2, (0,1)),
            
            spconv.SparseConv2d(128, 256, 5, stride=1, padding=2),
            nn.LeakyReLU(0.1),
            spconv.SparseMaxPool2d(2, 2, (0,1)),
            
            spconv.ToDense(),
            nn.Flatten(1,-1),
            nn.Dropout(p=0.2),
            nn.Linear(4096, 2048),
            nn.Linear(2048, 1024),
            nn.Linear(1024, 8),
        )

    def forward(self, x: torch.Tensor):
        x = spconv.SparseConvTensor.from_dense(x.reshape(-1, POINTS, 3, 1))
        y =  self.net(x)
        
        return y

In [5]:
X_train, X_test, y_train, y_test = parse_partial_dataset(getData, getAnnotation)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

train_loader = torch.utils.data.DataLoader(
    [[X_train[i], y_train[i]] for i in range(len(X_train))], shuffle=False, batch_size=40)
test_loader = torch.utils.data.DataLoader(
    [[X_test[i], y_test[i]] for i in range(len(X_test))], shuffle=False, batch_size=40)

args = lambda: None
args.fp16 = True
args.lr = 0.001
args.log_interval = 50

device = torch.device("cuda:0") if torch.cuda.is_available() else torch.device("cpu")
print(device)

def makeTestData():
    testData = []
    
    for i in range(10): 
        X_test, y_test = parse_random_dataset(getData, getAnnotation, True)
        test_loader = torch.utils.data.DataLoader(
            [[X_test[i], y_test[i]] for i in range(len(X_test))], shuffle=False, batch_size=40)
        
        testData.append(test_loader)
        
    return testData

def testTiming(model, testData):
    total_time = 0.0
    total_acc = 0.0
    for i in range(10): 
        test_loader = testData[i]

        testing_time = time.time()

        iou = test(args, model, device, test_loader)

        time_taken = time.time()  - testing_time

        total_time += time_taken
        total_acc += iou

        print("\tFor this mini round, time: ", time_taken, " iou ", iou)

    avg_time = total_time / 10
    avg_acc = total_acc / 10
    
    print("Time on avg: ", avg_time, " IOU on avg: ", avg_acc)
    
    return avg_time, avg_acc

def trainModel():
    model = Net().to(device)
    optimizer = optim.Adam(model.parameters(), lr=args.lr)
    
    train_start_time = time.process_time()

    for epoch in range(0, 40):
        train(args, model, device, train_loader, optimizer, epoch)

    train_time = time.process_time() - train_start_time
    
    print("Training done!  " + str(train_time))
    
    return train_time, model

def getResults():
    testData = makeTestData()
    
    train_timing_results = np.empty(20)
    test_timing_results = np.empty(20)
    test_acc_results = np.empty(20)

    for i in range(20):
        print("Round ", i)

        train_time, model = trainModel()
        train_timing_results[i] = train_time
        
        test_time, test_acc = testTiming(model, testData)
        test_timing_results[i] = test_time
        test_acc_results[i] = test_acc

    print("\n\nTesting done!\n")

    print("Training time: ", np.mean(train_timing_results))
    print("Testing time: ", np.mean(test_timing_results))
    print("Testing acc: ", np.mean(test_acc_results))
    
    return train_timing_results, test_timing_results, test_acc_results

getResults()

cuda:0
Round  0
Training done!  29.09375
	For this mini round, time:  0.3539755344390869  iou  tensor(0.7037, device='cuda:0')
	For this mini round, time:  0.37800025939941406  iou  tensor(0.7095, device='cuda:0')
	For this mini round, time:  0.36500120162963867  iou  tensor(0.7062, device='cuda:0')
	For this mini round, time:  0.3690226078033447  iou  tensor(0.7096, device='cuda:0')
	For this mini round, time:  0.37302470207214355  iou  tensor(0.7147, device='cuda:0')
	For this mini round, time:  0.39397525787353516  iou  tensor(0.7109, device='cuda:0')
	For this mini round, time:  0.39299917221069336  iou  tensor(0.7164, device='cuda:0')
	For this mini round, time:  0.3900003433227539  iou  tensor(0.7054, device='cuda:0')
	For this mini round, time:  0.4239990711212158  iou  tensor(0.7100, device='cuda:0')
	For this mini round, time:  0.41499948501586914  iou  tensor(0.7136, device='cuda:0')
Time on avg:  0.3854997634887695  IOU on avg:  tensor(0.7100, device='cuda:0')
Round  1
Train

	For this mini round, time:  0.39999961853027344  iou  tensor(0.6843, device='cuda:0')
	For this mini round, time:  0.41697072982788086  iou  tensor(0.6769, device='cuda:0')
	For this mini round, time:  0.38300037384033203  iou  tensor(0.6772, device='cuda:0')
	For this mini round, time:  0.4010004997253418  iou  tensor(0.6758, device='cuda:0')
	For this mini round, time:  0.40200090408325195  iou  tensor(0.6740, device='cuda:0')
	For this mini round, time:  0.3880007266998291  iou  tensor(0.6750, device='cuda:0')
Time on avg:  0.3858006477355957  IOU on avg:  tensor(0.6766, device='cuda:0')
Round  9
Training done!  25.515625
	For this mini round, time:  0.3679993152618408  iou  tensor(0.6991, device='cuda:0')
	For this mini round, time:  0.36503076553344727  iou  tensor(0.6990, device='cuda:0')
	For this mini round, time:  0.3550233840942383  iou  tensor(0.7000, device='cuda:0')
	For this mini round, time:  0.35899949073791504  iou  tensor(0.7016, device='cuda:0')
	For this mini round

	For this mini round, time:  0.4309995174407959  iou  tensor(0.7187, device='cuda:0')
Time on avg:  0.3920996904373169  IOU on avg:  tensor(0.7180, device='cuda:0')
Round  17
Training done!  25.765625
	For this mini round, time:  0.3670229911804199  iou  tensor(0.6819, device='cuda:0')
	For this mini round, time:  0.3690016269683838  iou  tensor(0.6842, device='cuda:0')
	For this mini round, time:  0.3740227222442627  iou  tensor(0.6807, device='cuda:0')
	For this mini round, time:  0.3680253028869629  iou  tensor(0.6853, device='cuda:0')
	For this mini round, time:  0.37798619270324707  iou  tensor(0.6935, device='cuda:0')
	For this mini round, time:  0.41900014877319336  iou  tensor(0.6861, device='cuda:0')
	For this mini round, time:  0.39999914169311523  iou  tensor(0.6861, device='cuda:0')
	For this mini round, time:  0.41299986839294434  iou  tensor(0.6803, device='cuda:0')
	For this mini round, time:  0.42000508308410645  iou  tensor(0.6844, device='cuda:0')
	For this mini round

(array([29.09375 , 25.4375  , 25.65625 , 25.59375 , 25.59375 , 25.546875,
        25.640625, 25.5     , 25.96875 , 25.515625, 25.484375, 25.84375 ,
        25.75    , 25.875   , 25.515625, 25.546875, 25.515625, 25.765625,
        25.65625 , 25.625   ]),
 array([0.38549976, 0.39409983, 0.38849838, 0.38560681, 0.38930209,
        0.39100003, 0.39350352, 0.38601563, 0.38580065, 0.38840497,
        0.39289632, 0.39629714, 0.38499339, 0.38720086, 0.38409553,
        0.38719885, 0.39209969, 0.39440887, 0.39349995, 0.41110506]),
 array([0.71000308, 0.68684906, 0.72538358, 0.72356617, 0.71352929,
        0.68822944, 0.73257381, 0.69967157, 0.67663479, 0.70102805,
        0.69281399, 0.70023555, 0.72471166, 0.72153276, 0.65365213,
        0.71742862, 0.71803069, 0.68464679, 0.71655506, 0.70101434]))